In [87]:
import numpy as np                    
import pandas as pd                   
import matplotlib.pyplot as plt       
%matplotlib inline
import seaborn as sns                 
from sklearn.model_selection import train_test_split                    
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor                      
import xgboost
from xgboost import XGBRegressor                                       
from sklearn.preprocessing import StandardScaler                         
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV                         
import warnings                       
warnings.filterwarnings("ignore")     

In [88]:
train = pd.read_csv("Traindata.csv")
test = pd.read_csv("DataSetRetail.csv")

In [89]:
display(train.head())
display(test.head())

,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17-01-2011,8091,216418,99.0375,111.8625,0,0,20
1,2,17-01-2011,8091,216419,99.0375,99.0375,0,0,28
2,3,17-01-2011,8091,216425,133.9500,133.9500,0,0,19
3,4,17-01-2011,8091,216233,133.9500,133.9500,0,0,44
4,5,17-01-2011,8091,217390,141.0750,141.0750,0,0,52


,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku
0,212645,16-07-2013,8091,216418,108.3000,108.3000,0,0
1,212646,16-07-2013,8091,216419,109.0125,109.0125,0,0
2,212647,16-07-2013,8091,216425,133.9500,133.9500,0,0
3,212648,16-07-2013,8091,216233,133.9500,133.9500,0,0
4,212649,16-07-2013,8091,217390,176.7000,176.7000,0,0


In [90]:
print("Training data shape (Rows, Columns):",train.shape)
print("Test data shape (Rows, Columns):",test.shape)

Training data shape (Rows, Columns): (150150, 9)
Test data shape (Rows, Columns): (13860, 8)


In [91]:
train_original=train.copy() 
test_original=test.copy()

In [92]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150150 entries, 0 to 150149
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   record_ID        150150 non-null  int64  
 1   week             150150 non-null  object 
 2   store_id         150150 non-null  int64  
 3   sku_id           150150 non-null  int64  
 4   total_price      150149 non-null  float64
 5   base_price       150150 non-null  float64
 6   is_featured_sku  150150 non-null  int64  
 7   is_display_sku   150150 non-null  int64  
 8   units_sold       150150 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 10.3+ MB


In [93]:
train.dtypes.value_counts()

int64      6
float64    2
object     1
Name: count, dtype: int64

In [94]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13860 entries, 0 to 13859
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   record_ID        13860 non-null  int64  
 1   week             13860 non-null  object 
 2   store_id         13860 non-null  int64  
 3   sku_id           13860 non-null  int64  
 4   total_price      13860 non-null  float64
 5   base_price       13860 non-null  float64
 6   is_featured_sku  13860 non-null  int64  
 7   is_display_sku   13860 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 866.4+ KB


In [95]:
test.dtypes.value_counts()

int64      5
float64    2
object     1
Name: count, dtype: int64

In [96]:
display(train.count())
display(test.count())

record_ID          150150
week               150150
store_id           150150
sku_id             150150
total_price        150149
base_price         150150
is_featured_sku    150150
is_display_sku     150150
units_sold         150150
dtype: int64

record_ID          13860
week               13860
store_id           13860
sku_id             13860
total_price        13860
base_price         13860
is_featured_sku    13860
is_display_sku     13860
dtype: int64

In [97]:
train.isnull().sum()

record_ID          0
week               0
store_id           0
sku_id             0
total_price        1
base_price         0
is_featured_sku    0
is_display_sku     0
units_sold         0
dtype: int64

In [98]:
test.isnull().sum()

record_ID          0
week               0
store_id           0
sku_id             0
total_price        0
base_price         0
is_featured_sku    0
is_display_sku     0
dtype: int64

In [99]:

train.describe()

,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
count,150150.000000,150150.000000,150150.000000,150149.000000,150150.000000,150150.000000,150150.000000,150150.000000
mean,106271.555504,9199.422511,254761.132468,206.626751,219.425927,0.095611,0.133200,51.674206
std,61386.037861,615.591445,85547.306447,103.308516,110.961712,0.294058,0.339792,60.207904
min,1.000000,8023.000000,216233.000000,41.325000,61.275000,0.000000,0.000000,1.000000
25%,53111.250000,8562.000000,217217.000000,130.387500,133.237500,0.000000,0.000000,20.000000
50%,106226.500000,9371.000000,222087.000000,198.075000,205.912500,0.000000,0.000000,35.000000
75%,159452.750000,9731.000000,245338.000000,233.700000,234.412500,0.000000,0.000000,62.000000
max,212644.000000,9984.000000,679023.000000,562.162500,562.162500,1.000000,1.000000,2876.000000


In [100]:
train['total_price'] = train['total_price'].fillna(train['total_price'].mean())  

In [101]:
train.drop(columns=['record_ID','week'], inplace=True, axis=1)
test.drop(columns=['record_ID','week'], inplace=True, axis=1)

In [102]:
display(train.head())
display(test.head())

,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,8091,216418,99.0375,111.8625,0,0,20
1,8091,216419,99.0375,99.0375,0,0,28
2,8091,216425,133.9500,133.9500,0,0,19
3,8091,216233,133.9500,133.9500,0,0,44
4,8091,217390,141.0750,141.0750,0,0,52


,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku
0,8091,216418,108.3000,108.3000,0,0
1,8091,216419,109.0125,109.0125,0,0
2,8091,216425,133.9500,133.9500,0,0
3,8091,216233,133.9500,133.9500,0,0
4,8091,217390,176.7000,176.7000,0,0


In [103]:

X = train.iloc[:,:-1]            


y = train['units_sold']          

In [119]:

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=4)

In [120]:

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(105105, 6) (45045, 6) (105105,) (45045,)


In [121]:
sc = StandardScaler()
X_train_Scaled = sc.fit_transform(X_train)
X_test_Scaled = sc.transform(X_test)

In [122]:
rf = RandomForestRegressor()
rf.fit(X_train_Scaled, y_train)

RandomForestRegressor()

In [123]:
y_pred = rf.predict(X_test_Scaled)

In [124]:
print("Train Score {:.2f} & Test Score {:.2f}".format(rf.score(X_train_Scaled, y_train),rf.score(X_test_Scaled, y_test)))

Train Score 0.93 & Test Score 0.72


In [125]:
y_pred_test = rf.predict(test)

In [126]:
y_pred_test.shape

(13860,)

In [127]:
xgb = xgboost.XGBRegressor()
xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [128]:

y_pred_xgb = xgb.predict(X_test)

In [129]:
y_pred_xgb.shape

(45045,)

In [130]:
print("Train Score {:.2f} & Test Score {:.2f}".format(xgb.score(X_train,y_train),xgb.score(X_test,y_test)))

Train Score 0.84 & Test Score 0.78


In [131]:
test.shape


(13860, 6)

In [132]:
y_pred_xgb = xgb.predict(test)

In [133]:
submission = pd.DataFrame({'record_ID': test_original['record_ID'], 'units_sold': y_pred_xgb})
submission.to_csv('FinalSubmission.csv', index=False)